## FINGER TAPPING ANALYSIS
##### Data collected from patients with neurodegenerative disorders as well as healthy controls

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import time
import sys
from tqdm import tqdm
import os
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import math
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
import keras
from keras.models import Model, model_from_yaml
from keras.layers import Input, Conv1D, Flatten, Dropout, MaxPooling1D, Dense
from keras.layers import Activation, BatchNormalization, concatenate
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
%matplotlib inline

# from IPython.display import Audio
# sound_file = './sound/beep.wav'

C:\Users\Vladislava\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Read the data

train = scipy.io.loadmat('TrainData.mat')
val = scipy.io.loadmat('ValData.mat')


In [3]:
Xtrain, Ytrain = train['X'], train['Y']
Xval, Yval = val['X'], val['Y']

### Arite, lez make a model

In [4]:
def CNNModel(inputShape, nConvLayers):
    
    input1 = Input(shape = inputShape)

    #convolutions
    x = input1
    
    
    for i in range(0,nConvLayers):
        
        nFilters = 128 if i>1 else 32*(i+1)
        inside = 3 if i>1 else 2
        for temp in range(0,inside):
            x = Conv1D(filters = nFilters,
                  kernel_size = 5,
                  padding = 'same',
                  strides = 1,
                  name = 'Conv1x5{}{}'.format(i,temp))(x)
            
            x = Activation('relu',name='ReLu{}{}'.format(i,temp))(x)
            x = BatchNormalization()(x)
            
        x = MaxPooling1D(2,
                      padding = 'same',
                      strides = 1,
                      name = 'MaxPooling1D{}'.format(i))(x)
    
    x = Dropout(0.5)(x)
    
    # Fully connected
    x = Flatten()(x)
    
    x = Dense(128)(x)
    x = Activation('relu', name = 'reLU_dense')(x)
    x = Dropout(0.6)(x)
    
    x = Dense(4)(x)
    x = Activation('softmax',name = 'Softmax')(x)

    m = Model(input1,x)
    return m


In [5]:
def saveModelTopology(model,modelName):
    model_json = model.to_yaml()
    with open(modelName+'.yaml', "w") as yaml_file:
        yaml_file.write(model_json)
    print("Saved model to {}.yaml".format(modelName))
    return


In [6]:
def defCallbacks(weightFile):
    
    checkpoint = ModelCheckpoint(weightFile,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only = True,
                                 save_weights_only = True,
                                 mode='max')
    early = EarlyStopping(monitor='val_acc',
                          patience = 15,
                          verbose = 1,
                          mode='max')
    return [checkpoint, early]
    

In [7]:
def fitModel(model, modelName, Xtrain, Ytrain, Xval, Yval, epochs,batch_size):
    
    tic = time.time()
    
    # make file name
    tm = time.gmtime()
    weightFile = 'BEST_WEIGHTS{}.{}.{}.{}.{}.{}.h5'.format(modelName,tm[2],tm[1],tm[0],tm[3]+1,tm[4])
    
    #define callbacks
    callbacks = defCallbacks(weightFile)
    
    # FIT THE MODEL
    history = model.fit(x = Xtrain, y = Ytrain,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data = (Xval,Yval),
                        callbacks = callbacks)
    toc = time.time()
    print("Finished training in {} min ({} h)".format(round((toc-tic)/60,2),round((toc-tic)/3600,2)))

    
    # Save the weights
    #model.save_weights(str(modelName)+'.h5') # ???????
    
    return history

In [8]:
# def evaluateModel(model, modelName, Xval, Yval):
#     tic = time.time()
#     predictions = model.predict(Xval)
#     toc = time.time()
#     print('Finished prediction in: {} min'.format(round((toc-tic)/60,2)))

#     print('Evaluating...')
#     score = model.evaluate(Xval,Yval,verbose=1)
#     print(score)

#     #Save that stuff too pls.
#     tm = time.gmtime()    
#     predictionFile = 'Predictions-{}.{}.{}.{}.{}.{}.csv'.format(modelName,tm[2],tm[1],tm[0],tm[3]+1,tm[4])

#     dfPredicted = pd.DataFrame(predictions)
#     dfPredicted = dfPredicted.idxmax(axis =1)
#     dfExpected = pd.DataFrame(Yval)
#     dfExpected = dfExpected.idxmax(axis =1)
#     df = pd.DataFrame({'Predicted':dfPredicted, 'Expected':dfExpected})
#     df.to_csv(predictionFile,index = False)
#     print('Saved predictions to: ', predictionFile)
    
#     bingos = sum(df['Predicted'] ==df['Expected'])
#     accRly = 100*bingos/df.shape[0]
#     print('Currently your actual accuracy on Xval is: {}%'.format(round(accRly,2)))
    
#     return accRly

In [9]:
histories = []
#modelDepths = []
accuracies = []
nConvLayers = 4   
epochs = 150
batch_size = 16

for i in range(4):
    
    model = CNNModel((Xtrain.shape[1],Xtrain.shape[2]),nConvLayers)
    opt = optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True)
    model.compile(optimizer = opt,
             loss='categorical_crossentropy',
             metrics = ['accuracy'])
    #model.summary()
   
    modelName = 'CNN055055'+ str(nConvLayers)
    #saveModelTopology(model,modelName)
    model.save(modelName+'CEO.h5')
    
    print('TRAINING...')

    history = fitModel(model,modelName, Xtrain, Ytrain, Xval, Yval, epochs,batch_size)
    histories.append(history.history)   
    #modelDepths.append(nConvLayers)
    
    

    
#     acc = evaluateModel(model, modelName, Xval, Yval)
#     accuracies.append(acc)
    
    
#     with open("hist.txt", "w") as f:
#         for h in histories:
#             f.write(str(h) +"\n")

#     with open("hist.txt", "r") as f:
#         for line in f:
#             histo.append(int(line.strip()))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2000, 6)           0         
_________________________________________________________________
Conv1x500 (Conv1D)           (None, 2000, 32)          992       
_________________________________________________________________
ReLu00 (Activation)          (None, 2000, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2000, 32)          128       
_________________________________________________________________
Conv1x501 (Conv1D)           (None, 2000, 32)          5152      
_________________________________________________________________
ReLu01 (Activation)          (None, 2000, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 2000, 32)          128       
__________

2719/2719 [==============================] - ETA: 11:30 - loss: 4.5776 - acc: 0.18 - ETA: 5:57 - loss: 4.3125 - acc: 0.1562 - ETA: 4:07 - loss: 4.3994 - acc: 0.166 - ETA: 3:11 - loss: 4.0995 - acc: 0.218 - ETA: 2:38 - loss: 3.8852 - acc: 0.225 - ETA: 2:15 - loss: 4.1439 - acc: 0.218 - ETA: 1:59 - loss: 4.0369 - acc: 0.223 - ETA: 1:47 - loss: 3.9445 - acc: 0.210 - ETA: 1:38 - loss: 3.9976 - acc: 0.215 - ETA: 1:30 - loss: 3.8832 - acc: 0.225 - ETA: 1:24 - loss: 3.7997 - acc: 0.233 - ETA: 1:19 - loss: 3.7496 - acc: 0.229 - ETA: 1:15 - loss: 3.6394 - acc: 0.245 - ETA: 1:11 - loss: 3.5025 - acc: 0.267 - ETA: 1:07 - loss: 3.5164 - acc: 0.266 - ETA: 1:04 - loss: 3.4483 - acc: 0.269 - ETA: 1:02 - loss: 3.3650 - acc: 0.272 - ETA: 59s - loss: 3.3364 - acc: 0.267 - ETA: 57s - loss: 3.3005 - acc: 0.27 - ETA: 55s - loss: 3.2785 - acc: 0.27 - ETA: 54s - loss: 3.2415 - acc: 0.27 - ETA: 52s - loss: 3.1765 - acc: 0.28 - ETA: 51s - loss: 3.0929 - acc: 0.29 - ETA: 49s - loss: 3.0951 - acc: 0.29 - ETA: 48

2719/2719 [==============================] - ETA: 35s - loss: 0.7859 - acc: 0.62 - ETA: 33s - loss: 0.9195 - acc: 0.62 - ETA: 32s - loss: 1.0237 - acc: 0.56 - ETA: 31s - loss: 1.0275 - acc: 0.59 - ETA: 30s - loss: 1.0826 - acc: 0.55 - ETA: 30s - loss: 1.0986 - acc: 0.56 - ETA: 29s - loss: 1.1050 - acc: 0.56 - ETA: 29s - loss: 1.0923 - acc: 0.54 - ETA: 29s - loss: 1.0828 - acc: 0.54 - ETA: 29s - loss: 1.0910 - acc: 0.53 - ETA: 28s - loss: 1.0784 - acc: 0.53 - ETA: 28s - loss: 1.0715 - acc: 0.54 - ETA: 28s - loss: 1.1044 - acc: 0.52 - ETA: 27s - loss: 1.1043 - acc: 0.53 - ETA: 27s - loss: 1.0959 - acc: 0.53 - ETA: 27s - loss: 1.0786 - acc: 0.53 - ETA: 27s - loss: 1.0770 - acc: 0.54 - ETA: 27s - loss: 1.0893 - acc: 0.53 - ETA: 27s - loss: 1.0927 - acc: 0.52 - ETA: 26s - loss: 1.0992 - acc: 0.51 - ETA: 26s - loss: 1.0971 - acc: 0.51 - ETA: 26s - loss: 1.1063 - acc: 0.50 - ETA: 26s - loss: 1.1205 - acc: 0.49 - ETA: 26s - loss: 1.1078 - acc: 0.51 - ETA: 25s - loss: 1.1078 - acc: 0.50 - ETA: 

2719/2719 [==============================] - ETA: 33s - loss: 1.7019 - acc: 0.43 - ETA: 31s - loss: 1.2098 - acc: 0.62 - ETA: 30s - loss: 1.0425 - acc: 0.70 - ETA: 30s - loss: 1.0086 - acc: 0.68 - ETA: 30s - loss: 0.9581 - acc: 0.70 - ETA: 29s - loss: 0.9058 - acc: 0.70 - ETA: 29s - loss: 0.8657 - acc: 0.73 - ETA: 29s - loss: 0.8715 - acc: 0.70 - ETA: 29s - loss: 0.9081 - acc: 0.66 - ETA: 28s - loss: 0.9034 - acc: 0.65 - ETA: 28s - loss: 0.8990 - acc: 0.64 - ETA: 28s - loss: 0.8844 - acc: 0.65 - ETA: 28s - loss: 0.8490 - acc: 0.66 - ETA: 28s - loss: 0.8269 - acc: 0.67 - ETA: 27s - loss: 0.8494 - acc: 0.66 - ETA: 27s - loss: 0.8346 - acc: 0.66 - ETA: 27s - loss: 0.8423 - acc: 0.66 - ETA: 27s - loss: 0.8599 - acc: 0.65 - ETA: 27s - loss: 0.8586 - acc: 0.65 - ETA: 26s - loss: 0.8673 - acc: 0.64 - ETA: 26s - loss: 0.8687 - acc: 0.64 - ETA: 26s - loss: 0.8607 - acc: 0.64 - ETA: 26s - loss: 0.8765 - acc: 0.64 - ETA: 26s - loss: 0.8734 - acc: 0.64 - ETA: 25s - loss: 0.8697 - acc: 0.64 - ETA: 

2624/2719 [===========================>..] - ETA: 32s - loss: 0.5755 - acc: 0.75 - ETA: 32s - loss: 0.6917 - acc: 0.78 - ETA: 31s - loss: 0.8075 - acc: 0.70 - ETA: 30s - loss: 0.7928 - acc: 0.71 - ETA: 30s - loss: 0.7780 - acc: 0.70 - ETA: 30s - loss: 0.8201 - acc: 0.67 - ETA: 29s - loss: 0.7554 - acc: 0.71 - ETA: 29s - loss: 0.7046 - acc: 0.73 - ETA: 29s - loss: 0.7100 - acc: 0.73 - ETA: 29s - loss: 0.7014 - acc: 0.72 - ETA: 28s - loss: 0.7001 - acc: 0.72 - ETA: 28s - loss: 0.7058 - acc: 0.72 - ETA: 28s - loss: 0.6975 - acc: 0.72 - ETA: 28s - loss: 0.6898 - acc: 0.73 - ETA: 27s - loss: 0.6680 - acc: 0.74 - ETA: 27s - loss: 0.6725 - acc: 0.73 - ETA: 27s - loss: 0.6517 - acc: 0.74 - ETA: 27s - loss: 0.6534 - acc: 0.74 - ETA: 27s - loss: 0.6608 - acc: 0.74 - ETA: 26s - loss: 0.6690 - acc: 0.74 - ETA: 26s - loss: 0.6586 - acc: 0.74 - ETA: 26s - loss: 0.6549 - acc: 0.74 - ETA: 26s - loss: 0.6395 - acc: 0.75 - ETA: 26s - loss: 0.6389 - acc: 0.75 - ETA: 25s - loss: 0.6312 - acc: 0.75 - ETA: 

KeyboardInterrupt: 

In [ ]:
def checkOutHistory(history):
    

    # plot Accuracy over Epochs
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train Acc','Val Acc'])
    plt.title('Accuracy for {} over epochs'.format(modelName))
    plt.show()
    print("Max val accuracy: ", max(history['val_acc']))
    print("Max train accuracy: ", max(history['acc']))

    # plot Loss over Epochs
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss','Val Loss'])
    plt.title('Loss for {} over epochs'.format(modelName))
    plt.show()
        
    return


In [ ]:
for h in histories:
    checkOutHistory(h)

In [ ]:
histories[0]